In [1]:
from keras_preprocessing import image
import tensorflow as tf

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
train_datagenerator = image.ImageDataGenerator(
    rotation_range = 0.18,
    shear_range = 0.18,
    zoom_range  = 0.18,
    fill_mode='reflect',
    rescale=1./255,
    horizontal_flip=True
)

validation_datagenerator = image.ImageDataGenerator(
    rescale=1./255
)

In [4]:
train = train_datagenerator.flow_from_directory(
    "Datasets/Train",
    target_size= (224,224),
    batch_size=32,
    # color_mode='grayscale',
    class_mode = "binary"
)

Found 6000 images belonging to 2 classes.


In [5]:
train.class_indices

{'covid': 0, 'normal': 1}

In [6]:
test = validation_datagenerator.flow_from_directory(
    "Datasets/Validation",
    target_size=(224,224),
    batch_size=30,
    class_mode='binary'
)

Found 1232 images belonging to 2 classes.


In [7]:
test.class_indices

{'covid': 0, 'normal': 1}

## MODEL


In [8]:
from keras.models import Sequential
from keras.layers import Conv2D,Dense,MaxPool2D,Dropout,Flatten,BatchNormalization

In [9]:
def create_model():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))  
    model.add(MaxPool2D())

    model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))  
    model.add(MaxPool2D())

    model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
    model.add(MaxPool2D())

    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    print(model.summary())
    return model

In [10]:
import os
import keras


In [11]:

from keras.callbacks import ModelCheckpoint,EarlyStopping,CSVLogger
filepath = "Datasets/best_model.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1,save_best_only=True,mode = "max")

early_stopping = EarlyStopping(monitor='val_loss',verbose=1,patience=3)

log_csv = CSVLogger('Datasets/model_logs.csv',separator=',',append=False)

callback_list = [checkpoint,early_stopping,log_csv]

In [12]:
if os.path.exists(filepath):
    model = keras.models.load_model(filepath)
else :
    model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 256)       2

In [13]:


history = model.fit_generator(
  train,
  steps_per_epoch = 3600//18,
  epochs = 100,
  verbose=1,
  validation_data=test,
  validation_steps=3600//18,

  callbacks=callback_list
)


c:\python39\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmph7v1sk0x.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmph7v1sk0x.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
200/200 [==============================] - 91s 371ms/step - loss: 0.6414 - accuracy: 0.6230 - val_loss: 0.4378 - val_accuracy: 0.7965

Epoch 00001: val_accuracy improved from -inf to 0.79654, saving model to Datasets\best_model.hdf5
Epoch 2/100
200/200 [==============================] - 70s 353ms/step - loss: 0.4823 - accuracy: 0.7652 - val_loss: 0.3738 - val_accuracy: 0.8544

Epoc

In [14]:
model.save("model.h5")

In [15]:
best_model = tf.keras.models.load_model("Datasets/best_model.hdf5")
model = tf.keras.models.load_model("model.h5")

In [16]:
[model_loss,model_accuracy] = model.evaluate_generator(test,verbose=1)
[best_model_loss,best_model_accuracy] = best_model.evaluate_generator(test,verbose=1)
if model_accuracy>best_model_accuracy:
    model.save("Datasets/model.h5")
    print("Saved New Model")

c:\python39\lib\site-packages\tensorflow\python\keras\engine\training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


42/42 [==============================] - 3s 59ms/step - loss: 0.1240 - accuracy: 0.9594
